# Генерація тексту

У цій роботі ми використаємо мовну модель для генерації тексту.

У класичної мовної моделі є два взаємопов'язані визначення:

1. Оцінити ймовірність вхідного тексту.
2. Видати ймовірнісний розподіл наступного слова для даного префіксу.

Для генерації тексту нам ідеально підходить друге визначення.

## Початок роботи

Будь ласка, заповніть поля `EMAIL`, `NAME` та `GROUP` нижче:

In [1]:
!pip install --quiet --ignore-installed https://static.lp-nlp.com/pypi/lpnlp-2025.10.14-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
################################################################################
# FILL-IN:
#-----------------------------------------------------------------------
EMAIL = "denys.koval.shi.2022@lpnu.ua"  # заповніть вашим значенням
################################################################################

import lpnlp

lab = lpnlp.start(email=EMAIL, lab="text_generation")

Удачі!


### Завантаження моделі

Тренування мовної моделі з нуля займає багато часу: від кількох годин або днів для маленьких та середніх моделей й аж до кількох місяців чи навіть років для великих. Звичайно, за рахунок розпаралелювання тренування на багатьох GPU, реальний час рідко буває більше місяця-двох.

Для цієї роботи (як і в реальному житті), ми візьмемо претреновану модель. Хтось натренував її за нас. В данному випадку, візьмемо GPT-2 від компанії OpenAI.

Ця модель, як і безліч інших, зберігається на [HuggingFace Models](https://huggingface.co/models). Завантажити та працювати з нею зручно через бібліотеку [HuggingFace Transformers](https://github.com/huggingface/transformers/)

In [3]:
!pip install transformers

In [4]:
import transformers
import torch

In [5]:
model = transformers.AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Токенізація

Кожна модель має словник токенів, з яким вона тренувалася, та правила токенізації (розбиття тексту на токени). Обов'язково слід використовувати той самий словник та метод.

Бібліотека transformers вміє робити це для кожної підтримуваної моделі.

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Перевіримо токенізатор:

In [7]:
input_text = "Hello, LP NLP!"
token_ids = tokenizer.encode(input_text)
token_ids

[15496, 11, 18470, 399, 19930, 0]

In [8]:
tokenizer.convert_ids_to_tokens(token_ids)

['Hello', ',', 'ĠLP', 'ĠN', 'LP', '!']

Рідкісні слова будуть розбиті на підслова:

In [9]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("My name is Oleksiy Syvokon"))

['My', 'Ġname', 'Ġis', 'ĠOle', 'ks', 'iy', 'ĠSy', 'v', 'ok', 'on']

## Перевірка моделі -- один крок ітерації

Ми генеруватимемо текст в циклі токен за токеном, зліва направо. Але для початку розберімо, як виглядає один крок такого циклу.


На кожному кроці на вхід моделі подаємо ті токени, які вже згенеровано.

На першому кроці у нас ще нічого не згенеровано. Тому починаємо зі токену `<BOS>` ("begin of sentence"). В різних моделях він виглядає по-різному. Подивимося, що в GPT-2:


In [10]:
tokenizer.bos_token

'<|endoftext|>'

In [11]:
tokenizer.bos_token_id

50256

In [13]:
# Формуємо вхідний батч. Майже завжди для більшої ефективності
# нейронні мережі очікують на вхід кілька незалежних речень
# (або зображень у випадку з комп'ютерним зором)
#
# В нашій роботі ми завжди працюємо лише з одним реченням,
# тож розмір батча дорівнює одинці. Але все одно маємо
# оформити вхід як матрицю:
input_ = torch.LongTensor([[tokenizer.bos_token_id]])
input_

tensor([[50256]])

In [14]:
# Нарешті робимо крок генерації
output = model(input_, return_dict=True)

# На виході модель повертає вектор з logits -- ненормалізованими
# ймовірностями кожного токена в словнику
output.logits.shape

torch.Size([1, 1, 50257])

Чому маємо саме таку розмірність?

* Перша одиниця -- розмір батча, тобто кількість вхідних речень.
* Друга одиниця -- це довжина вхідної послідовності. На першому кроці ми подали лише один токен (bos_token)
* 50257 -- це розмір словника

In [15]:
tokenizer.vocab_size

50257

Поки що модель повернула не ймовірності, а просто якісь числа. Їм треба нормалізувати функцією softmax:

In [16]:
probs = torch.softmax(output.logits, dim=-1)

In [17]:
# Розмірність має збігатися з розміром словника
probs.shape

torch.Size([1, 1, 50257])

In [18]:
# Сума ймовірностей має дорівнювати 1.0
probs.sum()

tensor(1.0000, grad_fn=<SumBackward0>)

Ймовірності можуть ставати дуже маленькими та причиняти проблеми у зв'язку з обмеженою точністю float чисел. Тому прийнято працювати з логарифмами ймовірностей.

In [19]:
log_probs = torch.log_softmax(output.logits, dim=-1)

# Щоб перейти до звичайних ймовірностей, маємо зробити експоненціювання
log_probs.exp().sum()

tensor(1.0000, grad_fn=<SumBackward0>)

Кожному слову в словнику відповідає своя ймовірність бути побаченим
після заданого префікса. Префіксом у нас поки що був лише одни `bos_token`.


In [20]:
# Яка ймовірність, що речення почнеться зі слів "I", "red", "Why"?
for word in ("I", "red", "Why"):
    index = tokenizer.convert_tokens_to_ids(word)
    prob = log_probs[0, 0, index].exp()
    print(f"P({word}) = {prob}")

P(I) = 0.018320858478546143
P(red) = 0.00010652346827555448
P(Why) = 0.0008686637738719583


Тепер ми маємо ймовірностний розподіл по словнику. Можемо обрати слово, яке вважатимемо згенерованим. Тут можливі кілька стратегій, які ми розглянемо в наступних розділах.

## Greedy decoding

Найпростіший (але й не дуже цікавий) спосіб -- це завжди обирати токен з найбільшою ймовірністю:

In [21]:
next_token_id = log_probs.argmax()
next_token_id

tensor(198)

In [22]:
probs[-1, -1, 198]

tensor(0.0623, grad_fn=<SelectBackward0>)

In [23]:
tokenizer.convert_ids_to_tokens([next_token_id])  # Наш перший згенерований токен

['Ċ']

Зберемо код докупи та додамо цикл. В циклі ми продовжуватимемо генерувати текст токен за токеном, поки не настане одна з двох умов:
1. Модель видала спецальний токен `eos_token` (end of sentence)
2. Довжина згенерованого тексту перевищила певний поріг `max_len`

У хорошої моделі в більшості випадків має спрацьовувати перша умова зупинки. Проте іноді модель може впасти в безкінчений цикл. Щоб цьому запобігти, маємо другу умову.

In [28]:
tokenizer.eos_token

50257

In [32]:
@torch.no_grad()
def greedy_decode(model, tokenizer, max_len=50):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_, return_dict=True)
        probs = output.logits[0, -1]
        # Обираємо токен, що має найбільшу ймовірність
        token_index = probs.argmax()

        # Зупиняємося, якщо досягли кінця
        if token_index == tokenizer.eos_token_id:
          break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    return result


generated_token_ids = greedy_decode(model, tokenizer)

Маємо список згенерованих індексів токенів, який починається ось так:

In [33]:
generated_token_ids[:10]

[50256, 198, 464, 717, 640, 314, 2497, 262, 649, 2196]

Перетворимо їх в текст:

In [34]:
tokens = tokenizer.convert_ids_to_tokens(generated_token_ids)
tokens[:10]

['<|endoftext|>',
 'Ċ',
 'The',
 'Ġfirst',
 'Ġtime',
 'ĠI',
 'Ġsaw',
 'Ġthe',
 'Ġnew',
 'Ġversion']

In [35]:
generated_text = "".join(tokens)
generated_text

'<|endoftext|>ĊTheĠfirstĠtimeĠIĠsawĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcited.ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠto'

In [36]:
lab.checkpoint("greedy decode", generated_text)

Відповідь правильна ✅



'<|endoftext|>ĊTheĠfirstĠtimeĠIĠsawĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcited.ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠto'

### Примітка: Byte-pair encoding (BPE)

Наша модель використовує subword токенізацію, а саме byte-pair encoding (BPE). В сучасному NLP це найрозповсюдженіший спосіб токенізації. Детально можете подивитися в [цьому відео](https://www.youtube.com/watch?v=tOMjTCO0htA).

Для наших цілей зараз важливо, що BPE заміняє пробіли на спеціальні Unicode-символи "Ġ". Серед інших моделей широко поширений варіант "▁" (зверніть увагу, це не звичайний символ підкреслення "_"). Щоб отримати чистий текст, треба виконати наступну заміну:

In [37]:
def bpe_decode(s):
    result = s.replace("Ġ", " ")
    result = result.replace("Ċ", "\n")
    return result

bpe_decode(generated_text)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

Але краще довіритися токенізатору й зробити цю роботу за нас:

In [38]:
tokenizer.convert_tokens_to_string(tokens)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

## Generic decoding function

Обирати слово з найбільшою ймовірністю -- не найкращий варіант для генерації тексту хоча б тому, що він завжди детерміновано призводить до однієї послідовності. Нижче ми подивимося на цікавіші альтернативи.

Цикл генерації залишиться той самий, що і в `greedy_decode()`. Відрізнятися буде лише один рядок -- той, в якому ми приймали рішення, яке слово обрати. Для зручності, винесемо цей рядок в окрему функцію. Ця функція прийматиме на вхід ймовірностний розподіл по словнику і повертає обраний токен.

In [39]:
def greedy_choice(probs):
    return probs.argmax()

Функцію генерації також трохи переробимо.

По-перше, додамо параметр `sample_fn` -- це має бути функція, яка обирає слово з ймовірностного розподілу, наприклад, `greedy_choice`.

По-друге, для зручності виконуватимемо BPE декодинг у середині функції генерації.

In [81]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, max_len=50, bpe_decode=True):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_, return_dict=True)
        probs = torch.softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(probs)     # <---------------- цей рядок змінено

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
          break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result


generate(model, tokenizer, greedy_choice)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

## Simple sampling

Перший альтернатива -- це sampling. Тепер ми обираємо наступний токен випадково, але пропорційно до ймовірностей.

In [54]:
def simple_sample(probs):
    return torch.multinomial(probs, num_samples=1)[0]

# Згенеруємо 5 речень
for i in range(1, 6):
    result = generate(model, tokenizer, simple_sample)
    print(f"#{i}: {result}")
    print()

#1: <|endoftext|>Writing an internship is difficult, so what I consider an internship is orientation range courses such as R&R or quantitative learning classes. You must be able to write and graduate motivation writing while simultaneously being able to work with candidates and knowledge suppliers to agree

#2: <|endoftext|>Info crafts cough to crawl paddle and glass bottle Edit

Crust and rip up in archive After

Crust and rip up in archive

Starts off like Firewall

Irregular rhyme

DOS stops to cough

#3: <|endoftext|>ICS booming in UK? highlights gap between labour productivity gap in caring fumble case

Updated

Canada's labour productivity is on the rise as enrolled working mothers look to earn proportionally less than also offering job opportunities.

The

#4: <|endoftext|>Swarche Khoury is usually recognized as a member of Iran's foreign service who received the Medal of Valor from the Israeli Defense Minister in Tehran in September 2012. While on a visit to the U.S. in October

## Sampling with temperature

Ми також можемо впливати на генерацію параметром температури softmax.

Більші значення температури призводять до того, що різниця між ймовірностями токенів зменшується, тобто розподіл стає більш рівномірним. На практиці це означає, що менш ймовірні варіанти обиратимуться частіше і згенерований текст може бути цікавішим. Однак якщо продовжувати піднімати температуру, то текст спочатку втратить зв'язність, далі почнуть розпадатися слова та граматичність.

Менші значення температури змінюють розподіл таким чином, що основна ймовірніста маса припадає на невелику кількість топових токенів. При температурі 0 вся ймовірність дістанеться одному токену й семплінг перетвориться на greedy decoding.

Згенеруємо тексти з різною температурою:

In [62]:
for temperature in (0.1, 0.3, 0.5, 0.8, 1.0, 1.25, 1.5, 2.0, 3.0, 5.0):

    def sample_with_temp(probs):
        logits = torch.log(probs + 1e-9)
        scaled_logits = logits / temperature
        scaled_probs = torch.softmax(scaled_logits, dim=-1)
        return torch.multinomial(scaled_probs, num_samples=1)[0]

    print(f"Sampling with temperature={temperature}")
    result = generate(model, tokenizer, sample_with_temp, max_len=15)
    print(result)
    print()

Sampling with temperature=0.1
<|endoftext|>
The first time I saw the new "The Walking Dead" trailer

Sampling with temperature=0.3
<|endoftext|>The first of two new series from the series, "The Last of

Sampling with temperature=0.5
<|endoftext|>
The NHL's new expansion team, the Chicago Blackhawks, will be

Sampling with temperature=0.8
<|endoftext|>* Forward that message to all readers to inform them that our 'FH

Sampling with temperature=1.0
<|endoftext|>


r. Payne maj. lit., is professor of Korean language

Sampling with temperature=1.25
<|endoftext|> his invasion Cerberus contacts players and proposes them

Just as his leadership

Sampling with temperature=1.5
<|endoftext|>YSTrien drove to plant on weakened yard soon after Viv Mountains solemn

Sampling with temperature=2.0
<|endoftext|>exciritomet Ips delays inception TillIDSovember Eye Kyle Affairs ONE happiest

Sampling with temperature=3.0
<|endoftext|>POL reformedaned Hof totally portable:\48handler los enc Grurarillian

Sa

In [64]:
# Яке значення `temperature` здається вам оптимальною?
lab.checkpoint("softmax temperature", 0.5)

Відповідь правильна ✅
Окей, добре


0.5

## Top-k sampling

In [65]:
def top_k_sampling(probs, k):
    topk = probs.topk(k)
    index = torch.multinomial(topk.values, num_samples=1)[0]
#     print(f"Top {k} words take {topk.values.sum():%} probability mass")
    return topk.indices[index]

In [66]:
k = 15
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 6):
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{i}: {result}")
    print()

#1: <|endoftext|>
In my previous blog posts, we discussed how to build a simple, yet robust and flexible

#2: <|endoftext|>.NET Framework 4.5.5 Release Candidate 1 (build.prop.release.1

#3: <|endoftext|>
There was a time when people were so excited to see the upcoming Super Mario Bros. for

#4: <|endoftext|>
It looks like we will probably see the launch of the first Windows 10 tablet in October for

#5: <|endoftext|>"We had a couple of people in the hospital with gunshot wounds to their face that looked like



## Nucleus (top-p) sampling

In [87]:
def nucleus_sampling(probs, max_p):
    sorted_probs = probs.sort(descending=True)
    cum_prob = 0.0
    sample_indices = []
    sample_probs = []
    for i in range(0, len(sorted_probs.values)):
        p = sorted_probs.values[i]
        cum_prob += p
        sample_probs.append(p)
        sample_indices.append(sorted_probs.indices[i])
        if cum_prob > max_p:
            break

    sample_probs = torch.tensor(sample_probs)
    index = torch.multinomial(sample_probs, num_samples=1)[0].item()
    return sample_indices[index]

In [88]:
for max_p in (0.0, 0.1, 0.3, 0.5, 0.6, 0.8, 1.0):
    sample_fn = lambda probs: nucleus_sampling(probs, max_p=max_p)
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{max_p}: {result}")
    print()

#0.0: <|endoftext|>
The first time I saw the new version of the game, I was so excited. I

#0.1: <|endoftext|>
The new version of the app is available for Android and iOS.

The new version

#0.3: <|endoftext|>This is a quick overview of the best strategies for building a strong foundation for your new business.

#0.5: <|endoftext|>Brief Summary:

The use of the "orifice" to detect sexual arousal and

#0.6: <|endoftext|>Why this might be so

by Michael Fox

BELGRADE: C


#0.8: <|endoftext|>1.

Turbine

Comes with a silver w/black. The

#1.0: <|endoftext|>What are you doing? Relax, friends, you're done while you wait for the day to



## Start from prompt

До цього моменту ми генерували текст з нуля. Однак значно кориснішим є задача генерації тексту від певного префікса або "підказки" -- в англійській мові це називається "prompt".

Prompt дозволить нам контролювати тематику згенерованого тексту або, як ми побачимо на іншій лекції, допоможе моделі виконувати різноманітні завдання.

In [89]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, prompt, max_len=50, bpe_decode=True):

    result = tokenizer.encode(prompt)     # <---- цей рядок додано

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result

In [90]:
k = 3
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 5):
    result = generate(model, tokenizer, sample_fn, prompt="One of my favorite things is")
    print(f"#{i}: {result}")
    print()


# Спробуйте змінити prompt на щось інше

#1: One of my favorite things is the way that you can make it look like it's a real thing.

I'm not saying you can't do it, but I'm not saying you can't.

I'm not saying you don

#2: One of my favorite things is when I'm on the phone and I see a bunch of people talking to each other and I'm like, "Hey, what's going on?" And they're like, "Oh, I'm talking to my wife

#3: One of my favorite things is that you can see how the whole thing is going to go.

The whole thing is going to start with the first two episodes.

I'm going to go through the whole thing and see how it goes

#4: One of my favorite things is the way that I can get my hands on the latest version of the game. It's a very nice game, and I'm very happy with the way the game is. I've been using the old version for about



# Zero-shot класифікація

Ми будемо досліджувати цю тему на окремій лекції. Але спробуємо застосувати мовну модель для задачі класифікації вже зараз. Хоча наша модель надто маленька й слабка для серйозного використання в zero-shot.


In [91]:
generate(model, tokenizer, greedy_choice, prompt="This film is awful! Actor play is terrible. The plot is dull. I would rate it as a").splitlines()[0]


'This film is awful! Actor play is terrible. The plot is dull. I would rate it as a B+.'

In [92]:
generate(model, tokenizer, greedy_choice, prompt="This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a").splitlines()[0]


'This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a 5.'

In [93]:
lab.answer("Готово!")

Відповідь правильна ✅
Сподіваюсь, ця лаба була не надто важкою :)


'Готово!'

# Real world

У цій роботі ми імплементували кілька методів декодінгу. Але, звичайно, все вже зроблено за нас.

Подивіться на параметри функції [generate()](https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate) з бібліотеки transformers. Багато з них мають виглядати знайомими.

Повний приклад використання:

In [94]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "I hope that in that practical class on text generation you"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, top_p=0.8, max_length=50)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I hope that in that practical class on text generation you will come to a similar understanding of the nature of computation and its application to a more general system.\n\nMy hope is that this course will enable students to make their own computational systems and applications']

# The End


In [95]:
lab.answer("Готово!")

Відповідь правильна ✅
Сподіваюсь, ця лаба була не надто важкою :)


'Готово!'